# Creating a poverty map using ShinyApps in R

### This notebook is a step-by-step guide to create a heatmap in ShinyApps using real population data and available poverty data in Sierra Leone. 

##### Before starting, make sure you have downloaded the following datasets
##### 1. Population data from www.worldpop.org
-  Click on Data > Population > Individual Countries
-  Search for country
-  Choose the most recent year available from the results and click “Data & Resources”
-  Scroll down and click “Download Entire Dataset”
-  *note- these files are normally about 80-90 MB- if your internet connection is poor, consider setting up the download to run overnight*

##### 2. Getting the Poverty Data from http://dhsprogram.com
-  If you haven’t already, register for access to DHS datasets at the DHS programme website
-  Once granted access, log in and download the following survey datasets for the most recent standard DHS survey from your chosen country, and save them in your working directory along with the population data:
    -  a. Individual recode > Stata dataset (This dataset only includes women aged 15-49)
    -  b. Geographic data > Shapefile

#### 3. Getting administrative data
-  Go to Humanitarian Data Exchange- search “[country name] boundary.” In the results, find the zipped shapefile with all administrative levels, download it, unzip it, and save it on your computer.

In [2]:
#I load the packages needed for the project
library(shiny)
library(haven)
library(leaflet)
library(raster)
library(leaflet.extras)
library(RColorBrewer)
library(viridis)
library(terra)
library(sf)
library(rgdal)


-  Open the 'Individual recode' Stata dataset. You now need to think about how you want to define “poor” in your population. If you’re not sure, we recommend defining all individuals in the bottom two wealth quintiles (Q1/2) as poor.You can do this by running the following code in R:

In [5]:
base <- read_dta("povertySL/poor.DTA")

-  "v190" is the DHS variable for wealth quintile. Every individual below the quintile will be considered as poor

In [ ]:
base$q2 = ifelse(base$v190 <3, 1,0) 

-  I'll create an average for poor respondents in DHS for each cluster (v001 is the DHS cluster variable)

In [ ]:
tab1 <- aggregate(base$q2, list(base$v001), mean)

-  Now that I've created the variable, let's save this as a .csv file

In [ ]:
write.csv(tab1, "poor.csv")

-  I now need to match this data with administrative data and location points. For this, I need to paste the lat and long from the DHS database for each cluster. It might be easier to do this using Excel: for this, I open the .dbf file, which contains coordinates for the clusters of DHS respondents. I copy and paste cluster locations to my database (poor.csv)
-  I upload the database I created containing the location for each cluster

In [6]:
base <- read.csv("povertySL/poor.csv",header= TRUE, sep= ",", na.strings = "N/A")

-  Getting administrative data for my selected country, in this case, Sierra Leone

In [7]:
states <- readOGR(dsn = "povertySL/sle_admbnda_adm2_1m_gov_ocha/sle_admbnda_adm2_1m_gov_ocha_20161017.shp", 
                  layer = "sle_admbnda_adm2_1m_gov_ocha_20161017",GDAL1_integer64_policy = FALSE, verbose= FALSE)

Warning message:
“OGR support is provided by the sf and terra packages among others”
Warning message:
“OGR support is provided by the sf and terra packages among others”
Warning message:
“OGR support is provided by the sf and terra packages among others”
Warning message:
“OGR support is provided by the sf and terra packages among others”
Warning message:
“OGR support is provided by the sf and terra packages among others”
Warning message:
“OGR support is provided by the sf and terra packages among others”


-  I need to create a raster image that will be collated to the countries' boundariesand that will show the population density image that I downloaded from populationdata.org

In [9]:
r <- raster("povertySL/sle_admbnda_adm2_1m_gov_ocha/sle_ppp_2020_constrained.tif")

#### Given the size and time it takes for R to create a raster image, I downgrade the quality so it becomes faster
r <- aggregate(r,10)

-  I now have prepared my raster image that will go into the map, let's do some design!<br> 
#### DESIGN<br> 
-  I set up the color palette for the population density raster

In [10]:
magma1<- rev(viridis(10))
pal <- colorNumeric(
  palette = magma1,
  domain = values(r),
  na.color = "transparent")
pal1 <- colorNumeric(palette="Spectral", domain = base$mean)



Pinpointing sites
-  First, I create an icon that will be pinpointing teams' location. For this I use a person icon

In [12]:
msicon <- makeIcon(
  iconUrl = "povertySL/clinic.png",
  iconWidth = 20, iconHeight = 20
)

-  I upload a database with my fictional sites

In [13]:
base1<- read.csv("povertySL/sites.csv", header = TRUE, sep = ",")

### Starting the ShinyApp

-  Define UI for application that shows a world map with a heat map for Sierra Leone

In [15]:
ui <- navbarPage(div(style='font-size: 14px;', "Sierra Leone Poverty Map"),
                 windowTitle = "Sierra Leone Poverty Map",
                 id="nav",
                 collapsible = TRUE,
                 tabPanel("Interactive Results",
                          div(class="outer",
                              tags$head(
                                includeCSS("povertySL/styles.css")
                              ),
                              tags$style(type = "text/css", "#map {height: calc(100vh - 0px) !important;}"),
                              leafletOutput("map"),
                              absolutePanel(id= "controls1", fixed = TRUE,
                                            draggable = TRUE, top = "85%", left = "1%", right = "1%", bottom = "auto",
                                            width = "97%", height = "15%",
                                            plotOutput("violences", width = "100%", height = "100%"))
                              )
                          )
                 )



-  Define server logic required to draw a histogram

In [16]:
server <- function(input, output) {
  
  
  
    output$map <- renderLeaflet(
      leaflet()%>%
        addTiles() %>%
        addProviderTiles(
          provider= 'Esri.WorldGrayCanvas',
          group = "ESRI") %>%
        addRasterImage(r,
                       colors= pal,
                       opacity= 0.8)%>%
        addHeatmap(lng=base$long,lat=base$lat,intensity= base$mean2,max=1,radius=40, blur=60) %>%
        addLegend(pal = pal1,
                  values = base$mean,
                  title= "Poverty",
                  labFormat = labelFormat(transform = function(x) sort(x, decreasing = TRUE))) %>%
        addMarkers(base1$long, base1$lat, icon = msicon)
    )
}



#### Run the application

In [ ]:
shinyApp(ui = ui, server = server)


Listening on http://127.0.0.1:5317

